#### VactionPy

##### Import Libraries and Load the Weather and Coordinates Data

In [128]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_key_main import geoapify_key

In [129]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,11.41,63,0,2.57,US,1699053973
1,1,waitangi,-43.9535,-176.5597,16.57,94,82,3.13,NZ,1699053963
2,2,bilibino,68.0546,166.4372,-12.94,94,100,1.62,RU,1699053970
3,3,luderitz,-26.6481,15.1594,15.87,81,0,5.30,NaN,1699054072
4,4,west island,-12.1568,96.8225,24.99,83,75,5.66,CC,1699053994


##### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [130]:
# %%capture --no-display
# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    size = "Humidity", 
    scale = 1,
    color = "City",
    alpha = 0.5  
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

##### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [139]:
# Narrow down cities that fit criteria and drop any results with null values
city_ideal_df = city_data_df
#Ideal conditions Max Temp > 22C, Wind Speed < 5, and 0 cloudiness
city_ideal_df = city_ideal_df.loc[(city_ideal_df["Max Temp"] > 21) & (city_ideal_df["Max Temp"] < 27)]
city_ideal_df = city_ideal_df.loc[city_ideal_df["Wind Speed"] < 5]
city_ideal_df = city_ideal_df.loc[city_ideal_df["Cloudiness"] == 0]


# Drop any rows with null values
city_ideal_df = city_ideal_df.dropna()

city_ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
106,106,dickinson,29.4608,-95.0513,22.31,78,0,4.12,US,1699054302
165,165,mhamid,29.8200,-5.7200,22.48,28,0,1.38,MA,1699054315
212,212,winnemucca,40.9730,-117.7357,21.03,30,0,1.54,US,1699054329
213,213,i-n-salah,27.1935,2.4607,26.86,12,0,2.82,DZ,1699054329
272,272,selma,36.5708,-119.6121,26.86,29,0,0.00,US,1699054343
315,315,mersa matruh,31.3525,27.2453,21.11,56,0,0.00,EG,1699054359
319,319,santa rosalia,27.3167,-112.2833,26.32,34,0,2.90,MX,1699054360
330,330,zouerate,22.7187,-12.4521,22.98,34,0,3.44,MR,1699054363
360,360,salalah,17.0151,54.0924,25.05,73,0,1.54,OM,1699054370
376,376,san sebastian de la gomera,28.0916,-17.1133,23.54,56,0,3.60,ES,1699054374


##### Step 3: Create a new DataFrame called `hotel_df`.

In [140]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(city_ideal_df, columns = ["City", "Country", "Lat", "Lng", "Humidity", "Hotel Name"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
106,dickinson,US,29.4608,-95.0513,78,
165,mhamid,MA,29.8200,-5.7200,28,
212,winnemucca,US,40.9730,-117.7357,30,
213,i-n-salah,DZ,27.1935,2.4607,12,
272,selma,US,36.5708,-119.6121,29,


##### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [141]:
# Set parameters to search for a hotel
radius = 10000 
params = {
        "categories" : "accommodation.hotel",
        "apiKey" : geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")   
# Display sample data
hotel_df

Starting hotel search


dickinson - nearest hotel: Days Inn by Wyndham Dickinson TX
mhamid - nearest hotel: Hotel Kasbah Azalay
winnemucca - nearest hotel: The Overland Hotel
i-n-salah - nearest hotel: باجودة
selma - nearest hotel: Best Western
mersa matruh - nearest hotel: صقور
santa rosalia - nearest hotel: Sol y Mar
zouerate - nearest hotel: فندق تيرس زمور
salalah - nearest hotel: Muscat International Hotel
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
banes - nearest hotel: No hotel found
timimoun - nearest hotel: أناف
port isabel - nearest hotel: Queen Isabel Inn
adrar - nearest hotel: No hotel found
leesville - nearest hotel: Surestay Hotel By Best Western Leesville
awjilah - nearest hotel: No hotel found
port augusta - nearest hotel: Hotel Commonwealth
al kharijah - nearest hotel: Kharga Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
106,dickinson,US,29.4608,-95.0513,78,Days Inn by Wyndham Dickinson TX
165,mhamid,MA,29.8200,-5.7200,28,Hotel Kasbah Azalay
212,winnemucca,US,40.9730,-117.7357,30,The Overland Hotel
213,i-n-salah,DZ,27.1935,2.4607,12,باجودة
272,selma,US,36.5708,-119.6121,29,Best Western
315,mersa matruh,EG,31.3525,27.2453,56,صقور
319,santa rosalia,MX,27.3167,-112.2833,34,Sol y Mar
330,zouerate,MR,22.7187,-12.4521,34,فندق تيرس زمور
360,salalah,OM,17.0151,54.0924,73,Muscat International Hotel
376,san sebastian de la gomera,ES,28.0916,-17.1133,56,Hotel Villa Gomera


##### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [142]:
# %%capture --no-display

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    size = "Humidity", 
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"],
    alpha = 0.5  
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)